In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/univ/4/nlp/gpt

Mounted at /content/drive
/content/drive/My Drive/univ/4/nlp/gpt


In [ ]:
import pandas as pd
import numpy as np
from metrics import exact_match_score, accuracy, f1, qa_f1_score

# CSV 파일 읽기
csv_file = 'baseline2_gpt_result.csv'
df = pd.read_csv(csv_file)

# query와 response를 group으로 묶음
df['group'] = df.index // 5

first_rows = df.groupby('group').first().reset_index(drop=True)

# first_rows에서 응답과 정답 추출
preds = first_rows['response'].tolist()
ground_truths = first_rows['laws'].tolist()

# F1 점수 계산
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")



F1 Score: 3.4454718889332914%


In [ ]:
!pip install nltk rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f3acd0a7a5251862afd3bea991dc7a6fa4bd75066e70e3517c36898f0877ee4e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

bleu_scores = [sentence_bleu([gt.split()], pred.split()) for pred, gt in zip(preds, ground_truths)]
bleu_score = 100 * np.mean(bleu_scores)
print(f"BLEU Score: {bleu_score}%")
smoothing = SmoothingFunction().method4

bleu1_scores = [sentence_bleu([gt.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]
bleu2_scores = [sentence_bleu([gt.split()], pred.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]

bleu1_score = 100 * np.mean(bleu1_scores)
bleu2_score = 100 * np.mean(bleu2_scores)

print(f"BLEU-1 Score: {bleu1_score}%")
print(f"BLEU-2 Score: {bleu2_score}%")

# ROUGE 점수 계산
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, gt in zip(preds, ground_truths):
    scores = scorer.score(gt, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

rouge1_score = 100 * np.mean(rouge1_scores)
rouge2_score = 100 * np.mean(rouge2_scores)
rougeL_score = 100 * np.mean(rougeL_scores)

print(f"ROUGE-1 Score: {rouge1_score}%")
print(f"ROUGE-2 Score: {rouge2_score}%")
print(f"ROUGE-L Score: {rougeL_score}%")

BLEU Score: 0.02256173685139998%
BLEU-1 Score: 2.27137996338391%
BLEU-2 Score: 0.8847435387800409%
ROUGE-1 Score: 13.16850948583363%
ROUGE-2 Score: 3.7568660968713194%
ROUGE-L Score: 12.435965406733434%


In [ ]:
import pandas as pd
import numpy as np
from metrics import exact_match_score, accuracy, f1
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
# CSV 파일 읽기
alpha_csv_file = 'AlphaMist_total_result.csv'
alpha_df = pd.read_csv(alpha_csv_file)
csv_file = 'baseline2_gpt_result.csv'
df = pd.read_csv(csv_file)
alpha3_csv_file='baseline3_AlphaMist_result.csv'
alpha3_df=pd.read_csv(alpha3_csv_file)


# 유효한 인덱스 필터링
valid_indices = alpha_df[alpha_df['답변'].notna() & alpha_df['RAG  답변'].notna()&alpha3_df['response'].notna()].index

csv_file = 'baseline2_gpt_result.csv'
df = pd.read_csv(csv_file)
df['group'] = df.index // 5
first_rows = df.groupby('group').first().reset_index(drop=True)
preds = first_rows.loc[valid_indices, 'response'].tolist()
ground_truths = first_rows.loc[valid_indices, 'laws'].tolist()
# F1 점수 계산
f1_score = f1(preds, [[gt] for gt in ground_truths])
print(f"F1 Score: {f1_score}%")
bleu_scores = [sentence_bleu([gt.split()], pred.split()) for pred, gt in zip(preds, ground_truths)]
bleu_score = 100 * np.mean(bleu_scores)
print(f"BLEU Score: {bleu_score}%")
smoothing = SmoothingFunction().method4

bleu1_scores = [sentence_bleu([gt.split()], pred.split(), weights=(1, 0, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]
bleu2_scores = [sentence_bleu([gt.split()], pred.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing) for pred, gt in zip(preds, ground_truths)]

bleu1_score = 100 * np.mean(bleu1_scores)
bleu2_score = 100 * np.mean(bleu2_scores)

print(f"BLEU-1 Score: {bleu1_score}%")
print(f"BLEU-2 Score: {bleu2_score}%")


scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, gt in zip(preds, ground_truths):
    scores = scorer.score(gt, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

rouge1_score = 100 * np.mean(rouge1_scores)
rouge2_score = 100 * np.mean(rouge2_scores)
rougeL_score = 100 * np.mean(rougeL_scores)

print(f"ROUGE-1 Score: {rouge1_score}%")
print(f"ROUGE-2 Score: {rouge2_score}%")
print(f"ROUGE-L Score: {rougeL_score}%")

F1 Score: 3.6385913280113624%
BLEU Score: 7.215208660986937e-155%
BLEU-1 Score: 2.4026599390950736%
BLEU-2 Score: 0.9066903900971566%
ROUGE-1 Score: 15.14548443771365%
ROUGE-2 Score: 4.6669014289605%
ROUGE-L Score: 14.223383777475329%


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_